In [19]:
import requests
from bs4 import BeautifulSoup
from time import sleep

In [2]:
def parse_url(url):
    r = requests.get(url)
    if not r.status_code == requests.codes.ok:
        r.raise_for_status()
    r.encoding = r.apparent_encoding
    bs = BeautifulSoup(r.text, 'html.parser')
    return bs

In [3]:
def parse_file(file):
    with open(file) as f:
        bs = BeautifulSoup(f, 'html.parser')
    return bs

In [4]:
def strip_generator(generator):
    list_ = []
    for v in generator:
        if isinstance(v, str):
            list_.append(v.strip())
    return list_

# スポットURL一覧を取得

In [5]:
url = 'https://www.jalan.net/kankou/pro_003/g2_U2/'
bs = parse_url(url)

In [6]:
# path = '../locals/jalan.html'
# bs = parse_file(path)

In [7]:
items = bs.findAll('div', class_='item-listContents')
items

[<div class="item-listContents">
 <div class="item-photo">
 <div class="item-mainImg"><img alt="ねねさんの吹割の滝の投稿写真1" height="171" src="//cdn.jalan.jp/jalan/img/3/kuchikomi/0173/KM/39900_0000173243.jpg" width="228"/></div>
 <ul class="item-thumbs">
 <li><img alt="ベベッチさんの吹割の滝の投稿写真1" height="56" src="//cdn.jalan.jp/jalan/img/3/kuchikomi/0333/KXS/fd222_0000333455.jpg" width="74"/></li>
 <li><img alt="seanさんの吹割の滝の投稿写真1" height="56" src="//cdn.jalan.jp/jalan/img/5/kuchikomi/0005/KXS/2f0e6_0000005063.jpg" width="74"/></li>
 <li><img alt="まみぃままさんの吹割の滝の投稿写真1" height="56" src="//cdn.jalan.jp/jalan/img/0/kuchikomi/0520/KXS/8a45c_0000520529_1.jpg" width="74"/></li>
 </ul>
 </div>
 <div class="item-info">
 <div class="item-innerBtnArea">
 <ul>
 <li class="btnWannagoSpotGray">
 <a data-bookmarktype="2" data-editkbn="1" data-odktype="1" data-page="1" data-spoteventid="guide000000151053" onclick="javascript:doWannagoWent($(this));return false;">行きたい</a>
 </li>
 <li class="btnWentSpotGray">
 <a data-bookma

In [8]:
urls = ['https:' + item.findAll('a')[2]['href'] for item in items]
urls

['https://www.jalan.net/kankou/spt_guide000000151053/',
 'https://www.jalan.net/kankou/spt_guide000000184651/',
 'https://www.jalan.net/kankou/spt_guide000000150978/',
 'https://www.jalan.net/kankou/spt_guide000000164423/',
 'https://www.jalan.net/kankou/spt_guide000000202105/',
 'https://www.jalan.net/kankou/spt_guide000000185267/',
 'https://www.jalan.net/kankou/spt_guide000000195909/',
 'https://www.jalan.net/kankou/spt_guide000000193354/',
 'https://www.jalan.net/kankou/spt_guide000000193353/',
 'https://www.jalan.net/kankou/spt_guide000000192183/',
 'https://www.jalan.net/kankou/spt_guide000000186613/',
 'https://www.jalan.net/kankou/spt_guide000000175433/',
 'https://www.jalan.net/kankou/spt_guide000000164421/']

# スポット詳細ページをパース

In [9]:
url = 'https://www.jalan.net/kankou/spt_guide000000151053/'
bs = parse_url(url)

In [10]:
# path = '../locals/jalan_detail.html'
# bs = parse_file(path)

In [11]:
name = bs.find('h1', class_='detailTitle').string
name

'吹割の滝'

In [12]:
gen = bs.find('div', id='aboutArea').find('p').strings
description = '\n'.join(strip_generator(gen))
print(description)

1936年に天然記念物及び名勝に指定された、東洋のナイアガラとも呼ばれる幅30ｍ高さ７ｍの「吹割の滝」。
群馬県沼田市にあり、利根川水系。
滝から吹き上がる水しぶきが河床を割くように流れているように見えたことで、この名がつけられました。
また、「竜宮の椀」の伝説がある滝で、滝壷は竜宮に通じているとも言われています。
その昔、村人が祝儀の度に、お椀やお膳を借りたいとお願いをした手紙を滝壺に投げ入れたといわれたが、
一度だけ借りたものを返し忘れてしまったことで、それ以来は二度とお椀やお膳を借りれなくなったといわれています。
※天候により遊歩道が通れない場合もあるので要注意。


In [13]:
map = bs.find('div', id='detailMap-canvas')
lat, lon = map['data-lat'], map['data-lng']
print(lat, lon)

36.7023529 139.20641


In [14]:
gs = bs.find('dl', class_='c-genre').findAll('div', class_='dropdownCurrent')[1].find('a').string
gs

'滝・渓谷'

In [15]:
image = 'http:%s' % bs.find('div', class_='detailGallery_box galleryArea-image').find('img')['src']
image

'http://cdn.jalan.jp/jalan/img/3/kuchikomi/0173/KL/39900_0000173243.jpg'

In [16]:
access_text = bs.find('table', class_='basicInfoTable').findAll('tr')[1].find('td').string.strip()
access_text

'(1)関越道沼田ICよりR120経由、日光方面へ30分'

In [20]:
url = 'https://www.jalan.net/kankou/pro_003/g1_22/'
urls = []
pager = [url] + ['%spage_%d/' % (url, i) for i in range(2, 6)]
for page in pager:
    try:
        bs = parse_url(page)
        items = bs.findAll('div', class_='item-listContents')
        urls.extend(['https:' + i.findAll('a')[2]['href'] for i in items])
        sleep(5)
    except requests.exceptions.HTTPError as e:
        break
urls

['https://www.jalan.net/kankou/pro_003/g1_22/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_2/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_3/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_4/', 'https://www.jalan.net/kankou/pro_003/g1_22/page_5/']


['https://www.jalan.net/kankou/spt_14205ab2050129880/',
 'https://www.jalan.net/kankou/spt_08364ab2040005938/',
 'https://www.jalan.net/kankou/spt_13110ab2042041292/',
 'https://www.jalan.net/kankou/spt_09206ab2040006069/',
 'https://www.jalan.net/kankou/spt_12226ab2010146434/',
 'https://www.jalan.net/kankou/spt_08201ab2030005124/',
 'https://www.jalan.net/kankou/spt_13101aj2200023392/',
 'https://www.jalan.net/kankou/spt_08220ab2010003284/',
 'https://www.jalan.net/kankou/spt_13106ab2030004433/',
 'https://www.jalan.net/kankou/spt_guide000000187656/',
 'https://www.jalan.net/kankou/spt_12465ab2050034231/',
 'https://www.jalan.net/kankou/spt_guide000000179765/',
 'https://www.jalan.net/kankou/spt_09202cc3310040305/',
 'https://www.jalan.net/kankou/spt_08201cb3410089210/',
 'https://www.jalan.net/kankou/spt_09206ab2030004392/',
 'https://www.jalan.net/kankou/spt_09383ab2040006204/',
 'https://www.jalan.net/kankou/spt_14214ab2010002020/',
 'https://www.jalan.net/kankou/spt_11363ab201011